In [1]:
!pip install timm
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 419.3 kB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-mwarza1v
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-mwarza1v
  Resolved https://github.com/openai/CLIP.git to commit d50d76daa670286dd6cacf3bcd80b5e4823fc8e1
  Preparing metadata (setup.py) ... - done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369409 sha256=b2c1868c1a5deb26adab7ec8bd278fd27c2c4c514a0beff42c8be2b854be1343
  Stored in directory: /tmp/pip-ephem-wheel-cache-rapq50i9/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [2]:
from transformers import BeitModel, BeitFeatureExtractor
import torch.nn as nn
import torch
from PIL import Image
import requests
import timm
from torchvision import transforms
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import urllib
import os
import numpy as np
import clip
from clip.clip import _download, _MODELS

In [3]:
model_path = _download(_MODELS['ViT-L/14@336px'], os.path.expanduser("~/.cache/clip"))
with open(model_path, 'rb') as opened_file:
    clip_vit_l14_336 = torch.jit.load(opened_file, map_location="cuda").visual.eval()

100%|████████████████████████████████████████| 891M/891M [00:08<00:00, 108MiB/s]


In [4]:
class CLIP_Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone_clip = clip_vit_l14_336
#         self.resize = transforms.Resize(size=(248, 248), interpolation=transforms.InterpolationMode.BICUBIC, max_size=None, antialias=None)
#         self.crop = transforms.CenterCrop(size=(224, 224))
#         self.normilize = transforms.Normalize(mean=[0.5000, 0.5000, 0.5000], std=[0.5000, 0.5000, 0.5000])
        self.avgpool1d = nn.AdaptiveAvgPool1d(64)
        
    def forward(self, x):
        x = transforms.functional.resize(x,size=[336, 336])
        x = x/255.0
        x = transforms.functional.normalize(x, mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
        x = self.backbone_clip(x.half())
        x = self.avgpool1d(x)
#         embedding = torch.nn.functional.normalize(embedding, p=2.0, dim = 1)
        return x

In [5]:
model = CLIP_Encoder().to('cuda').eval()

In [6]:
url, filename = ("http://images.cocodataset.org/val2017/000000039769.jpg", "000000039769.jpg")
urllib.request.urlretrieve(url, filename)
img = Image.open(filename).convert('RGB')
convert_to_tensor = transforms.Compose([transforms.PILToTensor()])
input_tensor = convert_to_tensor(img)
input_batch = input_tensor.unsqueeze(0)
print(img.size)
with torch.no_grad():
    embedding = torch.flatten(model(input_batch.cuda())[0]).cpu().data.numpy()
print(embedding.shape)

(640, 480)
(64,)


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py:1110: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /usr/local/src/pytorch/aten/src/ATen/native/BinaryOps.cpp:601.)
  return forward_call(*input, **kwargs)


In [7]:
np.sqrt(sum([x*x for x in embedding]))
# 3.195444323984148
# 1.000000004405024

1.5544198347312401

In [8]:
saved_model = torch.jit.script(model)
saved_model.save('saved_model.pt')

In [9]:
url, filename = ("http://images.cocodataset.org/val2017/000000039769.jpg", "000000039769.jpg")
urllib.request.urlretrieve(url, filename)
img = Image.open(filename).convert('RGB')
convert_to_tensor = transforms.Compose([transforms.PILToTensor()])
input_tensor = convert_to_tensor(img)
input_batch = input_tensor.unsqueeze(0)
print(img.size)
with torch.no_grad():
    embedding = torch.flatten(saved_model(input_batch.cuda())[0]).cpu().data.numpy()
print(embedding.shape)

(640, 480)
(64,)


In [10]:
np.sqrt(sum([x*x for x in embedding]))

1.5544198347312401

In [11]:
from zipfile import ZipFile
with ZipFile('submission.zip', 'w') as zip:
    zip.write('./saved_model.pt', arcname='saved_model.pt')